In [2]:
import pandas as pd
import phate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import matplotlib.lines as mlines

# Load the features and labels
# Replace with your actual file paths
features = pd.read_csv("path\\to\\your\\featuresforphate.csv", header=None)
labels = pd.read_csv("path\\to\\your\\labelsforphate.csv", header=None)

# Convert labels into a list for easy access
label_list = labels[0].tolist()

# Initialize PHATE for 3D reduction
phate_operator = phate.PHATE(n_components=3)

# Transform your high-dimensional data into 3D using PHATE
Y_phate = phate_operator.fit_transform(features)

# Create a color map based on unique labels with shades
color_map = {
    'WT': '#78B16A',  # Bright Green
    'C228T': '#DC864A',  # Orange
    'C250T': '#79CED9'  # Blue
}

# Create 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


# Create legend handles manually
legend_handles = [mlines.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_map[label], markersize=2) for label in color_map.keys()]

# Set transparency using the alpha parameter
alpha_value = 0.2

# Plot points with colors based on their labels
scatter_objects = []
for i in range(Y_phate.shape[0]):
    sc = ax.scatter(Y_phate[i, 0], Y_phate[i, 1], Y_phate[i, 2], c=color_map[label_list[i]], alpha=alpha_value)
    scatter_objects.append(sc)

# Add legend
ax.legend(legend_handles, color_map.keys(), loc='best')

# Function to update the plot for animation
def update(num):
    ax.view_init(30, 30 + num)
    return scatter_objects

# Create animation
ani = animation.FuncAnimation(fig, update, frames=range(0, 360, 1), blit=True)

# Save the animation as a high-quality GIF
ani.save('IFC_Both_All3_rotating_phate.gif', writer='imagemagick', fps=20, dpi=1000)

# Show plot (optional)
plt.show()
fig.savefig("my_3Dphate_All3_plot.png", dpi=1000)

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np

# Load features and labels
# Replace with your actual file paths
features_path = "path\\to\\your\\latentfeatures.csv"
labels_path = "path\\to\\your\\labelsforphate.csv"
features = pd.read_csv(features_path, header=None)
labels = pd.read_csv(labels_path, header=None)

# Convert features to float and labels to a Series
features = features.astype(float)
labels = labels.squeeze()

# Create an AnnData object with features
adata = sc.AnnData(features)
adata.obs['labels'] = labels.values

# Debugging: Check unique labels and set colors accordingly
unique_labels = adata.obs['labels'].unique()
print("Unique labels:", unique_labels)

# Define colors for each label manually to ensure consistency
label_colors = {'WT': '#79CED9', 'TPM': '#DF884B'}  # Example: Light Blue for 'WT', Orange for 'TPM'
adata.uns['labels_colors'] = [label_colors[label] for label in unique_labels if label in label_colors]

# Run PCA and compute neighbors
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, use_rep='X_pca')

# Compute diffusion maps and DPT
sc.tl.diffmap(adata)

# Set the root cell for DPT to a 'WT' cell
wt_indices = adata.obs.index[adata.obs['labels'] == 'WT']
if len(wt_indices) > 0:
    adata.uns['iroot'] = adata.obs_names.get_loc(wt_indices[0])
else:
    raise ValueError("No 'WT' cells found in the data.")

sc.tl.dpt(adata)

# Perform clustering using Louvain algorithm and run PAGA
sc.tl.leiden(adata)
sc.tl.paga(adata, groups='leiden')

# Plot PAGA graph using defined colors
sc.pl.paga(adata, color='labels')

# Optional: Plot DPT if available
if 'dpt_pseudotime' in adata.obs:
    sc.pl.scatter(adata, basis='diffmap', color='dpt_pseudotime', title='Diffusion Pseudotime')

# Save the outputs to an .h5ad file
output_path = "output_path.h5ad"  # Replace with your desired output path
adata.write(output_path)